# <font color='#2B4865'>**Hugging Face 🤗 Transformers Tutorial III**</font>

---
### Natural Language Processing
Date: Jan 11, 2023

Last Updated: Nov 30, 2023

Author: Lorena Calvo-Bartolomé (lcalvo@pa.uc3m.es)

Version 1.0

---
This notebook is based on the [Hugging Face course](https://huggingface.co/course/chapter1/1) and documentation available at the Hugging Face website.

It constitutes the third and last tutorial notebook on the usage of Hugging Face libraries as well as its application for solving a series of NLP tasks. In particular, we will be covering how to carry out Text Generation.

---

<font color='#E0144C'>**For this notebook's execution, we highly encourage you to use Google Colaboratory. While for the inference part it is not necessary, you will highly speed up the execution if you make use of a GPU. For doing so, follow the following steps:**</font>

<font color='#E0144C'>**1. Connect to hosted runtime**</font>

<font color='#E0144C'>**2. Enable GPU setting by clicking Edit -> Notebook Settings -> Select GPU in Hardware Acceleration Tab -> Save**</font>

## <font color='#2B4865'>Installing necessary packages, imports and auxiliary functions</font>

In [ ]:
# Install necessary packages
import importlib, os

necessary_packages = ['transformers[sentencepiece]', 'datasets', 'colored', 'evaluate', 'accelerate']
def import_missing(packages):
  for p in packages:
    try:
      mod = importlib.import_module(p)
      print(f"Package {p} already installed!")
      packages.remove(p)
    except ModuleNotFoundError:
      print(f"Installing package {p}")
      with open("requirements.txt", 'w') as f:
        f.write("\n".join(str(i) for i in packages))
  if os.path.isfile("requirements.txt"):
    %pip install --quiet -r "requirements.txt"

import_missing(necessary_packages)

In [ ]:
%pip install -q typing-extensions==4.5.0 kaleido ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers cohere openai tiktoken pydantic-core

In [ ]:
# Common imports
import os
import numpy as np
import pandas as pd
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import scipy
from colored import Fore, Back, Style
import torch
import json

# Figures plotted inside the notebook
%matplotlib inline
# High quality figures
%config InlineBackend.figure_format = 'retina'

# Figues size
plt.rcParams['figure.figsize'] = [8, 6]

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore',module='gradio')

In [ ]:
# To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# For fancy table Display
%load_ext google.colab.data_table

In [ ]:
# Google Drive integration

# Libraries to work with Google Drive and the file system
from google.colab import drive
import os, sys

# Drive is mounted
drive.mount("/content/drive")

# Current directory is shown
print(os.getcwd())

# We change to work directory
directory_path = "/content/drive/MyDrive/Colab Notebooks/proyecto_td/reference_code/Hugging_Face_Transformers_Tutorial_III_students/"  # Define directory_path here
if not os.path.exists(directory_path):
  os.makedirs(directory_path)
  print(f"Directory created: {directory_path}")
os.chdir(directory_path)

## <font color='#2B4865'>**4. Text Generation**
---
</font>

Text Generation is the task of producing new text. These models can, for instance, fill in incomplete text or paraphrase.

Some use cases include:

* **Story Generation**: By receiving an input like "*Once upon a time*", a story generation model can create a story based on those words as we saw with GPT-2.
* **Code Generation**: We can train a text generation model on code from scratch to help with repetitive coding tasks.

* **Text-to-Text Generation Models**, which are trained to learn the mapping between a pair of texts (e.g. translation from one language to another). They are trained with multi-tasking capabilities, they can accomplish a wide range of tasks, including summarization, translation, and text classification.

##### <font color='#2B4865'>**Demo**</font>

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, pipeline

#https://huggingface.co/spaces/lvwerra/codeparrot-generation

title = "CodeParrot Generator 🦜"
description = "This is a subspace to make code generation with [CodeParrot](https://huggingface.co/lvwerra/codeparrot), it is used in a larger [space](https://huggingface.co/spaces/loubnabnl/Code-generation-models-v1) for model comparison. For more flexibilty in sampling, you can find another demo for CodeParrot [here](https://huggingface.co/spaces/lvwerra/codeparrot-generation)."
example = [
    ["def print_hello_world():", 8, 0.6, 42],
    ["def get_file_size(filepath):", 40, 0.6, 42],
    ["def count_lines(filename):", 40, 0.6, 42],
    ["def count_words(filename):", 40, 0.6, 42]]
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small", low_cpu_mem_usage=True)


def code_generation(gen_prompt, max_tokens, temperature=0.6, seed=42):
    set_seed(seed)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    generated_text = pipe(gen_prompt, do_sample=True, top_p=0.95, temperature=temperature, max_new_tokens=max_tokens)[0]['generated_text']
    return generated_text


iface = gr.Interface(
    fn=code_generation,
    inputs=[
        gr.Textbox(lines=10, label="Input code"),
        gr.inputs.Slider(
            minimum=8,
            maximum=256,
            step=1,
            default=8,
            label="Number of tokens to generate",
        ),
        gr.inputs.Slider(
            minimum=0,
            maximum=2,
            step=0.1,
            default=0.6,
            label="Temperature",
        ),
        gr.inputs.Slider(
            minimum=0,
            maximum=1000,
            step=1,
            default=42,
            label="Random seed to use for the generation"
        )
    ],
    outputs=gr.Textbox(label="Predicted code", lines=10),
    examples=example,
    layout="horizontal",
    theme="peach",
    description=description,
    title=title
)
iface.launch()

##### <font color='#2B4865'>**Architecture for approaching the task**</font>

The most popular models for this task are GPT-based models (such as GPT-2). Since these models are trained on unlabeled data, we just need plain text to train our own model to generate a wide variety of documents, from code to stories. Regarding the Text-to-Text Generation Models, its most popular variants are T5, T0 and BART.

##### <font color='#2B4865'>**Evaluation metrics**</font>

**Typical metrics** for Text Generation are:

* **Cross Entropy**, which is a metric that calculates the difference between two probability distributions. Each probability distribution is the distribution of predicted words.
* **Perplexity**, which is the exponential of the cross-entropy loss. It evaluates the probabilities assigned to the next word by the model. Lower perplexity indicates better performance.

### <font color='#2B4865'>*4.1. Inference*</font>

Performing inference for a text generation task is pretty similar to what we saw in former tutorials for the studied tasks. For basic text generation tasks (i.e., stories or code generation), we need to provide the model to use in conjunction with the task identifier ``"text-generation"``:

In [ ]:
from transformers import pipeline

checkpoint_name = "gpt2"
text_generator = pipeline("text-generation",
                          model=checkpoint_name,
                          pad_token_id=tokenizer.eos_token_id)                  # Prevents warning during decoding

In [ ]:
prompt = "Once upon a time"
print(text_generator(prompt, max_length=50, do_sample=True, top_p=0.9))

The model is generating a random text with a total maximal length of $50$ tokens from context “*Once upon a time*”. To create text, the pipeline object invokes the method ``PreTrainedModel.generate()``. The default arguments for this method can be overridden in the pipeline.

Below is an example of text generation using directly the GPT2 model and its tokenizer, which includes calling ``generate()`` directly. Note that for the instantiation of the model alone, we utilize a ``CausalLM`` head.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

gpt2_tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
gpt2 = AutoModelForCausalLM.from_pretrained(checkpoint_name)
gpt2.config.pad_token_id = gpt2.config.eos_token_id

In [ ]:
tokenized_prompt = gpt2_tokenizer(prompt, return_tensors="pt")
output = gpt2.generate(**tokenized_prompt, max_length=50, do_sample=True, top_p=0.9)
print(f"{gpt2_tokenizer.batch_decode(output)[0]}")

The following code snippet creates a **pipeline for Code generation**, based on CodeParrot, similar to what is done above for the Gradio app.

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")

codeparrot = pipeline("text-generation", model="codeparrot/codeparrot-small")
outputs = codeparrot("def hello_world():", max_length=50, pad_token_id=tokenizer.eos_token_id)
print(outputs)

**Text-to-Text generation models** have a separate pipeline called ``text2text-generation``. This pipeline takes an input containing the sentence including the task and returns the output of the accomplished task.

In [ ]:
from transformers import pipeline

text2text_generator = pipeline("text2text-generation", model="t5-base")

# Question Answering
question = "What is 42 ?"
context = "42 is the answer to life, the universe and everything"
output = text2text_generator("question: " + question + " " + context)
print(f"{question} --> {output[0]['generated_text']}")

# Translation
source_sent = "I'm very happy"
output = text2text_generator("translate from English to French: " + source_sent)
print(f"{source_sent} --> {output[0]['generated_text']}")

### <font color='#2B4865'>*4.2. Decoding methods for language generation*</font>

There exist different decoding strategies that are used by **auto-regressive language generation models** (i.e., decoder models), being the currently most prominent one the following:

1. **Greedy search**: At each time step, it selects **the word with the highest probability as its next word**, which is given by:

  $$w_t = argmax_{w}P(w | w_{1:t-1})$$

    <br><center><img src="https://drive.google.com/uc?id=1TPmWTY4bljqyRAB0qTXJVtf3qQ5CRBbT" width="40%"></center><br>

  Starting from the word "*The*", the algorithm greedily chooses the next word of highest probability "*nice*" and so on, so that the final generated word sequence is *{"The", "nice", "woman"}*, with a probability of $0.5 \times 0.4 = 0.2$.

  The **way of generating text following a greedy search approach** is as follows:

  ```
  greedy_output = model.generate(input_ids, do_sample=False)
  ```

  The main drawbacks of this approach are:
  * The model starts repeating itself quickly
  * It misses high-probability words hidden behind a low-probability word, e.g., the word "*has*" with its high conditional probability of $0.9$ is hidden behind the word "*dog*", which has only the second-highest conditional probability, so that greedy search misses the word sequence *{"The", "dog", "has"}*.

2. **Beam search**: It reduces the risk of missing hidden high-probability word sequences by keeping the most likely ``num_beams`` of hypotheses at each time step and eventually choosing the hypothesis that has the overall highest probability. For example, with ``num_beams=2``:

    <br><center><img src="https://drive.google.com/uc?id=1Hi0dDz4PKe6dOHOZLbmPmP-1g6_7J1Ia" width="40%"></center><br>

  At time step 1, besides the most likely hypothesis *{"The", "nice"}*, beam search also keeps track of the second most likely one *{"The", "dog"}*. At time step 2, it finds that the word sequence *{"The", "dog, "has"}*, has with $0.36$ a higher probability than *{"The", "nice", "woman"}*, which has $0.2$, thus finding the most likely word sequence.

  The way of generating text following a beam search approach is by setting  ``num_beams > 1`` and ``early_stopping=True`` so that generation is finished when all beam hypotheses reached the EOS token. We can further improve the result to reduce repetitions of the same word sequences by **introducing n-grams**. The most common n-grams penalty makes sure that no n-gram appears twice by manually setting the probability of the next words that could create an already seen n-gram to 0, e.g.:
  ```
  beam_output = model.generate(input_ids, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, do_sample=False)
  ```

3. **Sampling**: It refers to **randomly picking the next word $w_t$ according to its conditional probability** distribution $w_t \sim P(w|w_{1:t-1})$:

    <br><center><img src="https://drive.google.com/uc?id=14rILPEf-c43F8vXc_o2hFcRNoWQlCvod" width="40%"></center><br>

  The word "*car*" is sampled from $P(w | \text{"The"})$, followed by sampling "*drives*" from $P(w | \text{"The"}, \text{"car"})$.

  We can activate sampling by setting ``do_sample=True``.

  One issue with sampling is that the models tend to produce incomprehensible nonsense. This may be solved by sharpening the conditional probability distribution by raising the likelihood of high-probability words and decreasing the likelihood of low-probability words, i.e.,  by lowering the softmax's so-called ``temperature``:
  
      <br><center><img src="https://drive.google.com/uc?id=1nz12FfdJIcf92LhS3pg6K5-PeUMQ_jrI" width="40%"></center><br>

  The conditional next word distribution of step $t=1$ becomes much sharper leaving almost no chance for the word "*car*" to be selected.

  In practice, the Transformers library provides two different sampling methods:

  *   **Top-k sampling:** The $K$ most likely next words are filtered and the probability mass is redistributed among only those $K$ next words. GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation:
  
    <br><center><img src="https://drive.google.com/uc?id=1PrJR5ZCe8-T4-1lsPt1tV6NCq7bSen95" width="70%"></center><br>

    Having set $K = 6$, in both sampling steps we limit our sampling pool to 6 words.

    We activate top-K by setting ``top_k`` to the size of the candidate set:

    ```
    sample_output = model.generate(input_ids, do_sample=True, top_k=50)
    ```

  *   **Top-p (nucleus) sampling:** Instead of sampling only from the most likely K words, Top-p sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability p. The probability mass is then redistributed among this set of words. This way, the size of the set of words (a.k.a the number of words in the set) can dynamically increase and decrease according to the next word's probability distribution:

    <br><center><img src="https://drive.google.com/uc?id=11bTzQPA_cNIPfJwaTMcwPMHfXF6aRYUn" width="70%"></center><br>

    Having set $p=0.92$, it picks the minimum number of words to exceed together $p=92\%$ of the probability mass ($V_{\text{top-p}}$). In the first example, this included the 9 most likely words, whereas it only has to pick the top 3 words in the second example to exceed 92%.
  
    We activate Top-p sampling by setting ``top_p``to 0 < top_p < 1 and ``top_k`` to 0:

    ```
    sample_output = model.generate(input_ids, do_sample=True, top_p=0.92, top_k=0)
    ```

###### **Exercise 4.1**

Take the prompt we defined in 4.1 (``prompt = "Once upon a time"``) and generate text with the GPT-2 model according to the different decoding methods explained above. When possible, carry out the fine-tuning of parameters (e.g., ``num_beams`` for greedy search, ``temperature`` for sampling methods, etc.).

Compare the results obtained from the different methods and comment below on the differences you observe.

In [ ]:
#<SOL>
# TODO: Define tokenizer, model and tokenize the promt we are going to use

#</SOL>

In [ ]:
#<SOL>
# Greedy search

#</SOL>

In [ ]:
#<SOL>
# Beam search

#</SOL>

In [ ]:
#<SOL>
# Top-k sampling

#</SOL>

In [ ]:
#<SOL>
# Top-p sampling

#</SOL>

### <font color='#2B4865'>*4.2. Fine-tuning*</font>

Fine-tuning a model for text generation is not different from what we have seen for previous tasks, just with a couple of peculiarities regarding preprocessing. Let's see the latter by using the [WikiText](https://huggingface.co/datasets/wikitext) language modeling dataset:

In [ ]:
from datasets import load_dataset

datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [ ]:
datasets

As checkpoint for this example, we will use the ``distilgpt2`` model:

In [ ]:
checkpoint_name = "distilgpt2"

A **standard preprocessing step** for both **auto-regressive and masked language modeling** is to **concatenate all the instances** and then **split the whole corpus into chunks of similar size**. This is quite different from our usual approach, where we simply tokenize individual examples. The reason for this is that if individual samples are too long, they may be truncated, resulting in the loss of information that may be valuable for our task.

The approach is then to first tokenize our corpus as usual, but without setting the ``truncation=True`` option in our tokenizer:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)

print(tokenizer.is_fast)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Having our text tokenized, the next step is to concatenate all our texts together and then split the result into small chunks of certain block size. The desired block size would be the maximum length with which our model was pretrained; yet, this might be too big to fit in the GPU RAM, so let's just work with $128$:

In [ ]:
print(tokenizer.model_max_length)

block_size = 128

The following function will carry out the grouping:

In [ ]:
def group_texts(examples):

  # Concatenate all texts
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])

  # We make all input samples have the same length equal to block_size
  # by dropping the small remainder, we could add padding if the model
  # supported it instead of this drop
  # You can customize this part to your needs.
  total_length = (total_length // block_size) * block_size

  # Split by chunks of max_len
  result = {
      k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
      for k, t in concatenated_examples.items()
  }

  # Create a new labels column as copy of the input_ids
  result["labels"] = result["input_ids"].copy()

  return result

Note that in the former function's last step, we are creating a new ``labels`` column which is a copy of the ``input_ids`` one. This is because the goal of masked language modeling is to predict randomly masked tokens in the input batch, and by adding a labels column, we give the ground truth for our language model to learn from.

In [ ]:
grouped_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=4)

Here we are sending a batch of $1,000$ examples to be treated by the preprocessing function; this means that we will drop the remainder to make the concatenated tokenized texts a multiple of ``block_size`` every $1,000$ examples. You can adjust this behavior with a higher batch size, but note that this will make the processing slower.

We can see that our datasets have changed: the samples now comprise blocks of block-size contiguous tokens that might span multiple of our original texts:

In [ ]:
tokenizer.decode(grouped_datasets["train"][1]["input_ids"])

Having the dataset fully preprocessed, we have everything we need to carry out the fine-tuning of our model. Yet, there are not many differences from what we have been doing until now.

In the next and last exercise of this notebook, your task will be to carry out the complete fine-tuning of a dataset of your choice (different from WikiTest) and its corresponding previous dataset preparation and preprocessing. Here are some instructions that you may find useful:

##### <font color='#2B4865'>**Some instructions on fine-tuning for text generation**</font>

You can mimic the training we carried out in the first notebook tutorial of Hugging Face Transformers, but note that:

- The model that you need to use is one with a language modeling head (``AutoModelForCausalLM``).
- There is no need for the ``compute_metrics`` function, as evaluation for text generation can be performed accurately based on the loss.
- As DataCollator, use an instance of the class ``DataCollatorForLanguageModeling``. Since we are performing Causal Language modeling, you need to set the argument ``mlm`` to False. Before creating the collator, you will need to set the tokenizer's PAD token as the EOS token. The following code shows how to make the instantiation:

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

###### **Exercise 4.2**

Your task here is to choose an autoregressive model from the Hub (try to use a different one from what we have been using in this tutorial) and a dataset to fine-tune for text generation (e.g., stories, code, etc.).

As **dataset**, you **cannot use one provided in the Hub**. This means that you will need to find one from other resources (e.g., Kaggle) or construct it by yourself (e.g., you may want to generate movie dialogues following the Star Wars style; for this, you can web scrap the dialogues from IMSDb). If necessary, carry out the cleaning of the dataset.

Once you have your dataset ready, proceed with its pre-processing as we have seen above, and fine-tune it following the specified guidelines. Choose the parameter that best fits your dataset.

Once the model is trained, you can check its performance via the following code snippet:

```python
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
```

After having the final fine-tuned model, create a pipeline and generate at least 10 different texts with it. For creating the pipeline, you will need to specify the model checkpoint, and the tokenizer and set the following argument:

```python
pad_token_id=tokenizer.eos_token_id
```

As the tokenizer for the pipeline, use that of the checkpoint you fine-tuned.

In [ ]:
#<SOL>

#</SOL>

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:



In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
from transformers import pipeline

chef = pipeline('text-generation', model= 'distilgpt2-finetuned-wikitext2/checkpoint-7000', tokenizer=checkpoint_name, pad_token_id=tokenizer.eos_token_id)